In [1]:
import os

from pathlib import Path

import pandas as pd

from star_parser import StarParser, StarTabDf

In [5]:
data_folder = Path(r'/mnt/DATA/andrea/AM_20210709_USP7_GMPS_P53Ub/cryolo/pick/STAR')
file_pattern = '*patch_aligned_doseweighted.star'
mic_prefix = ""
mic_prefix_remove = 0
mic_suffix_add = ""
mic_suffix_remove = ""
star_out = "cryolo_pick_all.star"
print(f'Data folder exists: {data_folder.exists()}')

Data folder exists: True


In [6]:

starfiles = list(data_folder.glob(f'{file_pattern}'))
try:
    assert starfiles
except AssertionError:
    print(f'No starfiles found in {data_folder}')
collated = []
for star in starfiles:
    try:
        data = StarParser(star).parse()['data_']
    except KeyError: #empty starfile
        continue
    mic_name = mic_prefix + star.name.replace(mic_suffix_remove,"").replace('.star',f'{mic_suffix_add}.mrc')
    data.fill_column('MicrographName', mic_name, create=True, store=True)
    # data.remove_columns(['ClassNumber',	'AnglePsi',	'AutopickFigureOfMerit'], store=True)
    new_order = [data.columns()[-1]] + data.columns()[:-1]
    data.reorder_columns(new_order, store=True)
    collated.append(data.to_df())

In [7]:
merged = pd.concat(collated)
collated_star = StarTabDf(merged)

In [8]:
collated_star.to_df()

,MicrographName,CoordinateX,CoordinateY
0,002304263855996947171_FoilHole_9253300_Data_92...,2020.4137229919434,3996.238605044782
1,002304263855996947171_FoilHole_9253300_Data_92...,4365.162048339844,4004.4308914393187
2,002304263855996947171_FoilHole_9253300_Data_92...,4958.126106262207,3995.2327294796705
3,002304263855996947171_FoilHole_9253300_Data_92...,720.1175880432129,3938.81580157578
4,002304263855996947171_FoilHole_9253300_Data_92...,1354.3257808685303,3944.0038911402225
...,...,...,...
1208171,006190234053886453365_FoilHole_9253839_Data_92...,5659.521446228027,148.926931142807
1208172,006190234053886453365_FoilHole_9253839_Data_92...,862.6143836975098,88.31869626045227
1208173,006190234053886453365_FoilHole_9253839_Data_92...,3538.5514068603516,92.36625337600708
1208174,006190234053886453365_FoilHole_9253839_Data_92...,4067.7878952026367,98.85941791534424


In [9]:
with open(data_folder/f'../{star_out}', 'w') as f:
    f.write(collated_star.to_star())